#HSB Project

In [7]:
from sklearn.model_selection import train_test_split
from scipy.stats import shapiro, kstest, probplot
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
test_path = '/content/drive/MyDrive/Project_AI_HSB/Vehicles_export_prices_scaled_stud_test_eng.xlsx'

test_df = pd.read_excel(test_path)

In [11]:
train_path = '/content/drive/MyDrive/Project_AI_HSB/Vehicles_export_prices_scaled_train_eng.xlsx'

train_df = pd.read_excel(train_path)

In [14]:
test_df.head(5)

,RPAKREP_VEHICLE_HKEY,COMPANY,OFFICE,OFFICE_MAIN_BRAND,CHASSIS_NUMBER,MANUFACTURER_SHORT,MANUFACTURER,VEHICLE_GROUP,VEHICLE_TYPE,MODEL_CODE,VARIANT,MILEAGE,OPERATING_HOURS,MILAGE_IN_FIELD,MILAGE_SALES,OPERATING_HOURS_SALES,RIM_KEY,COLOR_CODE,COLOR_CODE_NAME,COLOR,COLOR_TYPE,UPHOLSTERY_CODE,UPHOLSTERY,UPHOLSTERY_CODE_ALT,CERTIFICATE_TYPE,CERTIFICATE_TYPE_DATE,FACTORY_NUMBER,ENGINE_ID,ENGINE_TYPE,ENGINE_ID_ALT,TRANSMISSION,TRANSMISSION_TYPE,TRANSMISSION_ID,TRANSMISSION_SHORT,TRANSMISSION_NAME,RIMS,FRONT_TIRES,FRONT_TIRES_CONDITION,REAR_TIRES,REAR_TIRES_CONDITION,NUMBER_DOORS,NUMBER_SEATS,PERMITTED_TOTAL_WEIGHT,MAX_TRAILOR_LOAD,CURB_WEIGHT,YEAR_CONSTRUCTION,CONSTRUCTION_MONTH,NUMBER_AXLE,NUMBER_ENGINE_CYLINDER,REPAIR_RKZ,...,CCM,NUMBER_OWNERS,IS_USED_CAR,LEASING_CONTRACT_DATE,LEASING_START,LEASING_END,LEASING_MILAGE,PAINT_TYPE,FINANCING_TYPE,FINANCING_TYPE_NAME,KAT_VEHICLE,FUEL_TYPE,FUEL_TYPE_NAME,DRIVE_TYPE,DRIVE_TYPE_NAME,VEHICLE_MODEL_ID,VEHICLE_MODEL_ID_NAME,COMMISSION_TYPE,COMMISSION_TYPE_NAME,DEMONSTRATION_STATUS,PURCHASE_DATE,PURCHASE_BOOKING_DATE,PURCHASE_MILAGE,PURCHASE_OPERATION_HOURS,PRICE_LIST,DAY_OF_REGISTRATION,AT_LOCATION_SINCE,LAID_UP_TIME,SOLD_CUSTOMER_ID,SOLD_INVOICE_COSTUMER_ID,MILAGE_SALE,OPERATION_HOURS_SALE,SOLD_INVOICE_COSTUMER_ID2,CUSTOMER_TYPE,CUSTOMER_GROUP,CUSTOMER_GROUP_NAME,CUSTOMER_FEATURE,CUSTOMER_FEATURE_NAME,SALE_CUSTOMER_ID2,CUSTOMER_SALE_GROUP,CUSTOMER_SALE_GROUP_NAME,CUSTOMER_SALE_GROUP2,CUSTOMER_SALE_GROUP2_NAME,SCALED_CURRENT_VALUE,SCALED_INVENTURAL_VALUE,SCALED_REPORT_VALUE,SCALED_VALUATION_PRICE,SCALED_GUIDE_PRICE,SCALED_TOTAL_SALES_PRICE_BASIS,SCALED_TOTAL_SALE_PRICE
0,d72d3a1e1e58bbcdf875d3f131c9d795,24.0,56,BMW,SALLAAAG6GA786187,LAR,Land Rover,PKW,Discovery TD V6 HSE,9999,NaN,82000.0,0.0,1.0,82000.0,0.0,NaN,NaN,Nicht definiert,Santorini Black,N,NaN,Leather,N,NaN,0.0,NaN,NaN,6 Zyl.- V-Form Diese,NaN,NaN,8-Gang Automatik,NaN,Z,Automatic,NaN,NaN,0.0,NaN,0.0,5.0,5.0,0.0,0.0,2486.0,2016,1.0,0.0,6.0,0.0,...,2993.0,2.0,1.0,NaT,NaT,NaT,0.0,NaN,1,Kauf,N,3,Diesel,N,Nicht definiert,5.0,Gelaendewagen/Pickup,2,Gebrauchtwagen,NaN,2022-11-22,2022-11-22,82000.0,0.0,NaN,NaN,1230227.0,NaN,B10463,B10463,82000.0,0.0,B10463,K,81,W-Kunde/WA Fzg.fremd.gek.,EVPR,Endverbraucher private,B10463,81,W-Kunde/WA Fzg.fremd.gek.,EVPR,Endverbraucher private,0.064879,0.06582,0.950043,0.0,0.005172,0.024837,0.023686
1,347ce39240a79b971ca83f39ee574486,12.0,43,VOL,YV1UZH4V2R1891650,VOL,Volvo,XC6,XC60 T6 AWD Recharge Plus Blac,246H4TD0D1,NaN,10.0,0.0,1.0,10.0,0.0,NaN,717,Nicht definiert,Onyx Black Metallic,N,RG0R,Schwarz,N,NaN,0.0,NaN,5150542,4 Zylinder Reihe-Mot,NaN,NaN,8 Gang Automatik-Get,NaN,Z,Automatic,NaN,NaN,0.0,NaN,0.0,5.0,5.0,0.0,0.0,2150.0,2024,2.0,0.0,4.0,0.0,...,1969.0,0.0,0.0,NaT,NaT,NaT,0.0,NaN,3,Leasing,N,N,Andere,3,Allrad,5.0,Gelaendewagen/Pickup,1,Neuwagen,NaN,2024-01-26,2024-03-01,0.0,0.0,NaN,NaN,0.0,NaN,B31644,201991,10.0,0.0,201991,NaN,NaN,NaN,NaN,NaN,B31644,EVGW,NaN,EVGW,Endverbraucher Gewerblich,0.073861,0.00000,0.950043,0.0,0.008687,0.040592,0.034014
2,a072c802558432572b38513a750d038b,18.0,71,FOR,WF0FXXWPMFPL12286,FOR,Ford,KUG,Kuga 2.0 EcoBlue 4x4 Aut.,48,NaN,16414.0,0.0,1.0,0.0,0.0,NaN,NYE,Metallic,Lucid Red Metallic,1,NZ2,Sensico Leder-Optik/Stoff,1,NaN,0.0,NaN,NaN,4 Zyl. Reihe-Motor -,NW7,NaN,8-Gang Automatik,NXB,Z,Automatic,NaN,NaN,0.0,NaN,0.0,5.0,5.0,0.0,0.0,1769.0,2023,5.0,2.0,4.0,0.0,...,1996.0,1.0,1.0,NaT,NaT,NaT,0.0,NaN,NaN,NaN,N,3,Diesel,3,Allrad,5.0,Gelaendewagen/Pickup,1,Neuwagen,NaN,2023-05-04,2023-06-20,0.0,0.0,NaN,NaN,1230628.0,NaN,316346,61461,0.0,0.0,61461,NaN,NaN,NaN,NaN,NaN,316346,NaN,NaN,NaN,NaN,0.062778,0.00000,0.950043,0.0,0.006626,0.028136,0.021025
3,31e6df9e7667c4ca2bd302017b5caa62,10.0,31,OPE,WF0FXXWPMFPK55616,FOR,Ford,KUG,KUGA CX482E,48,NaN,10.0,0.0,1.0,0.0,0.0,NaN,NYA,Nicht definiert,WHITE PLATINUM Metallic,N,NZC,STOFF Anthrazit,N,NaN,0.0,NaN,NaN,1.5 EB 110/150,NWS,NaN,6 Gang Schalt-Getrie,NX1,Y,Handschaltung,NaN,NaN,0.0,NaN,0.0,5.0,5.0,2075.0,0.0,1564.0,2023,2.0,2.0,3.0,0.0,..

In [15]:
train_df.head(5)

,RPAKREP_VEHICLE_HKEY,COMPANY,OFFICE,OFFICE_MAIN_BRAND,CHASSIS_NUMBER,MANUFACTURER_SHORT,MANUFACTURER,VEHICLE_GROUP,VEHICLE_TYPE,MODEL_CODE,VARIANT,MILEAGE,OPERATING_HOURS,MILAGE_IN_FIELD,MILAGE_SALES,OPERATING_HOURS_SALES,RIM_KEY,COLOR_CODE,COLOR_CODE_NAME,COLOR,COLOR_TYPE,UPHOLSTERY_CODE,UPHOLSTERY,UPHOLSTERY_CODE_ALT,CERTIFICATE_TYPE,CERTIFICATE_TYPE_DATE,FACTORY_NUMBER,ENGINE_ID,ENGINE_TYPE,ENGINE_ID_ALT,TRANSMISSION,TRANSMISSION_TYPE,TRANSMISSION_ID,TRANSMISSION_SHORT,TRANSMISSION_NAME,RIMS,FRONT_TIRES,FRONT_TIRES_CONDITION,REAR_TIRES,REAR_TIRES_CONDITION,NUMBER_DOORS,NUMBER_SEATS,PERMITTED_TOTAL_WEIGHT,MAX_TRAILOR_LOAD,CURB_WEIGHT,YEAR_CONSTRUCTION,CONSTRUCTION_MONTH,NUMBER_AXLE,NUMBER_ENGINE_CYLINDER,REPAIR_RKZ,...,CCM,NUMBER_OWNERS,IS_USED_CAR,LEASING_CONTRACT_DATE,LEASING_START,LEASING_END,LEASING_MILAGE,PAINT_TYPE,FINANCING_TYPE,FINANCING_TYPE_NAME,KAT_VEHICLE,FUEL_TYPE,FUEL_TYPE_NAME,DRIVE_TYPE,DRIVE_TYPE_NAME,VEHICLE_MODEL_ID,VEHICLE_MODEL_ID_NAME,COMMISSION_TYPE,COMMISSION_TYPE_NAME,DEMONSTRATION_STATUS,PURCHASE_DATE,PURCHASE_BOOKING_DATE,PURCHASE_MILAGE,PURCHASE_OPERATION_HOURS,PRICE_LIST,DAY_OF_REGISTRATION,AT_LOCATION_SINCE,LAID_UP_TIME,SOLD_CUSTOMER_ID,SOLD_INVOICE_COSTUMER_ID,MILAGE_SALE,OPERATION_HOURS_SALE,SOLD_INVOICE_COSTUMER_ID2,CUSTOMER_TYPE,CUSTOMER_GROUP,CUSTOMER_GROUP_NAME,CUSTOMER_FEATURE,CUSTOMER_FEATURE_NAME,SALE_CUSTOMER_ID2,CUSTOMER_SALE_GROUP,CUSTOMER_SALE_GROUP_NAME,CUSTOMER_SALE_GROUP2,CUSTOMER_SALE_GROUP2_NAME,SCALED_CURRENT_VALUE,SCALED_INVENTURAL_VALUE,SCALED_REPORT_VALUE,SCALED_VALUATION_PRICE,SCALED_GUIDE_PRICE,SCALED_TOTAL_SALES_PRICE_BASIS,SCALED_TOTAL_SALE_PRICE
0,77c1af48604226d47b2ccf7be44df009,44.0,TA,TOY,WF01XXERK1JR43779,FOR,Ford,ECS,ECOSPORT 1.0 Titanium Eco,NaN,NaN,8600.0,0.0,1.0,8600.0,0.0,NaN,NaN,Nicht definiert,Magnetic-Grau Metallic,N,NaN,Teilleder,N,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Handschaltung,NaN,NaN,0.0,NaN,0.0,5.0,5.0,0.0,0.0,0.0,2018.0,10.0,0.0,3.0,0.0,...,998.0,1.0,1.0,NaT,NaT,NaT,0.0,NaN,1,Kauf,1,1,Bleifrei,N,Nicht definiert,0.0,Keine Zuteilung,2,Gebrauchtwagen,NaN,2024-02-29,2024-02-29,8600.0,0.0,15990,NaN,0.0,7.0,C38451,C38451,8600.0,0.0,C38451,K,EVPR,EV privat,EVPR,Endverbraucher private,C38451,EVPR,EV privat,EVPR,Endverbraucher private,0.052907,0.000000,0.950043,0.0,0.002975,0.011018,0.011321
1,39587e4f9324333232d2ffaa2f314e6d,12.0,44,VOL,YV1XZK7V5S2487523,VOL,Volvo,XC4,"XC40 B3 B DKG Core 120 kW, 5-t",536K7170F1,NaN,0.0,0.0,1.0,0.0,0.0,NaN,717,Nicht definiert,Onyx Schwarz-Metallic,N,R7C0,Grau,N,NaN,0.0,NaN,5363376,4 Zylinder Reihe-Mot,NaN,NaN,7 Gang Automatik-Get,NaN,Z,Automatic,NaN,NaN,0.0,NaN,0.0,5.0,5.0,0.0,0.0,1688.0,2025.0,7.0,0.0,4.0,0.0,...,1969.0,0.0,0.0,NaT,2024-07-25,2025-07-24,20000.0,NaN,3,Leasing,N,9,Andere,1,Vorderrad,5.0,Gelaendewagen/Pickup,1,Neuwagen,NaN,2024-07-03,2024-07-04,0.0,0.0,NaN,NaN,0.0,23.0,493187,40963,0.0,0.0,40963,K,GROS,Grossabnehmer,GROS,Grossabnehmer,493187,NaN,NaN,NaN,NaN,0.061605,0.000000,0.950043,0.0,0.006136,0.026411,0.019844
2,a91470879cf756368c1d0099a2fa939a,27.0,33,SKO,WF0DXXSK0RX009819,FOR,Ford,TOC,Tourneo Connect 2.0 EcoBl,26,NaN,1297.0,0.0,1.0,0.0,0.0,NaN,NaN,Nicht definiert,Ink Black Metallic,N,NaN,siehe Ausstattung,N,NaN,0.0,NaN,NaN,2.0 TDE 66KW/90 E4,C3,NaN,NaN,NaN,Y,Handschaltung,NaN,NaN,0.0,NaN,0.0,5.0,5.0,0.0,0.0,0.0,NaN,NaN,0.0,4.0,0.0,...,1968.0,0.0,0.0,2024-04-29,2024-06-05,2028-12-04,67500.0,NaN,3,Leasing,N,3,Diesel,N,Nicht definiert,4.0,Van/Kleinbus,1,Neuwagen,NaN,2024-05-23,2024-05-23,0.0,0.0,44462.2,N,0.0,13.0,553344,496814,0.0,0.0,496814,K,GROS,Grossabnehmer,GROS,Grossabnehmer,553344,EVPR,EV privat,EVPR,Endverbraucher private,0.063286,0.000000,0.950043,0.0,0.001378,0.022102,0.022053
3,61363a68ff967e7f0c4b80b46a55b141,33.0,K1,V,WVWZZZAWZPU075160,V,Volkswagen,POL,"POLO LIFE 1,0 L 59 KW (80 PS)",AE13GV,NaN,6020.0,0.0,1.0,6020.0,0.0,NaN,8E8E,Nicht definiert,Reflexsilber Metallic,N,EL,Titanschwarz/Titanschwarz,N,NaN,0.0,NaN,NaN,DSGD,NaN,V,UVN,NaN,N,Nicht definiert,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,

In [16]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99071 entries, 0 to 99070
Columns: 106 entries, RPAKREP_VEHICLE_HKEY to SCALED_TOTAL_SALE_PRICE
dtypes: datetime64[ns](5), float64(42), object(59)
memory usage: 80.1+ MB


In [17]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42460 entries, 0 to 42459
Columns: 106 entries, RPAKREP_VEHICLE_HKEY to SCALED_TOTAL_SALE_PRICE
dtypes: datetime64[ns](5), float64(41), object(60)
memory usage: 34.3+ MB


In [18]:
train_df.describe()

,COMPANY,MILEAGE,OPERATING_HOURS,MILAGE_IN_FIELD,MILAGE_SALES,OPERATING_HOURS_SALES,CERTIFICATE_TYPE_DATE,FRONT_TIRES_CONDITION,REAR_TIRES_CONDITION,NUMBER_DOORS,NUMBER_SEATS,PERMITTED_TOTAL_WEIGHT,MAX_TRAILOR_LOAD,CURB_WEIGHT,YEAR_CONSTRUCTION,CONSTRUCTION_MONTH,NUMBER_AXLE,NUMBER_ENGINE_CYLINDER,REPAIR_RKZ,OPTICAL_CONDITION,TECHNICAL_CONDITION,HORSEPOWER,KW,CCM,NUMBER_OWNERS,IS_USED_CAR,LEASING_CONTRACT_DATE,LEASING_START,LEASING_END,LEASING_MILAGE,PAINT_TYPE,VEHICLE_MODEL_ID,PURCHASE_DATE,PURCHASE_BOOKING_DATE,PURCHASE_MILAGE,PURCHASE_OPERATION_HOURS,AT_LOCATION_SINCE,LAID_UP_TIME,MILAGE_SALE,OPERATION_HOURS_SALE,SCALED_CURRENT_VALUE,SCALED_INVENTURAL_VALUE,SCALED_REPORT_VALUE,SCALED_VALUATION_PRICE,SCALED_GUIDE_PRICE,SCALED_TOTAL_SALES_PRICE_BASIS,SCALED_TOTAL_SALE_PRICE
count,99040.000000,9.904000e+04,99040.000000,99038.0,9.904000e+04,99040.000000,99004.0,99004.0,99004.0,99004.000000,99004.000000,99004.000000,99004.000000,99004.000000,74366.000000,70320.000000,99004.000000,99004.000000,99004.0,73107.000000,73057.0,99004.000000,99004.000000,99004.000000,99004.000000,99004.000000,10925,17106,21864,9.900400e+04,0.0,99004.000000,97001,96173,9.900400e+04,99004.0,9.900400e+04,99004.000000,9.900400e+04,99004.000000,9.900400e+04,99004.000000,99004.000000,99004.0,9.900400e+04,9.900400e+04,9.900400e+04
mean,22.375808,3.071296e+04,0.275535,1.0,2.439775e+04,0.275374,0.0,0.0,0.0,3.432962,3.439932,243.442265,77.284766,1075.761202,2020.570059,6.360922,0.580775,2.544826,0.0,0.000041,0.0,149.995606,110.888772,1522.064715,0.569836,0.459699,2023-04-30 23:38:46.736842240,2023-10-01 05:36:13.132234240,2025-09-27 16:03:33.391876864,7.046598e+03,NaN,3.010757,2023-09-04 06:56:46.752507648,2023-09-17 03:19:02.179197952,2.210805e+04,0.0,3.651454e+05,88.271939,2.422813e+04,0.275464,1.010119e+03,0.013350,0.944832,0.0,2.020125e+03,1.010080e+03,1.010078e+03
min,10.000000,0.000000e+00,0.000000,1.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000,-1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,-119.000000,0.000000,0.000000,2001-01-02 00:00:00,2001-01-01 00:00:00,2000-01-01 00:00:00,0.000000e+00,NaN,0.000000,2013-07-24 00:00:00,2013-08-05 00:00:00,0.000000e+00,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,2.980100e-04
25%,12.000000,1.000000e+01,0.000000,1.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2020.000000,3.000000,0.000000,0.000000,0.0,0.000000,0.0,109.000000,81.000000,999.000000,0.000000,0.000000,2023-02-28 00:00:00,2023-09-23 00:00:00,2024-12-27 00:00:00,0.000000e+00,NaN,0.000000,2023-04-19 00:00:00,2023-04-26 00:00:00,0.000000e+00,0.0,0.000000e+00,22.000000,0.000000e+00,0.000000,5.178134e-02,0.000000,0.950043,0.0,2.375720e-03,1.087726e-02,1.064459e-02
50%,18.000000,5.285000e+03,0.000000,1.0,3.003000e+03,0.000000,0.0,0.0,0.0,5.000000,5.000000,0.000000,0.000000,1345.000000,2022.000000,6.000000,0.000000,3.000000,0.0,0.000000,0.0,145.000000,110.000000,1498.000000,0.000000,0.000000,2023-09-05 00:00:00,2024-01-29 00:00:00,2026-04-04 00:00:00,0.000000e+00,NaN,3.000000,2023-09-19 00:00:00,2023-10-05 00:00:00,3.000000e+00,0.0,0.000000e+00,59.000000,1.000000e+02,0.000000,5.704270e-02,0.000000,0.950043,0.0,4.373960e-03,1.805227e-02,1.578809e-02
75%,33.000000,2.694025e+04,0.000000,1.0,2.400000e+04,0.000000,0.0,0.0,0.0,5.000000,5.000000,0.000000,0.000000,1748.000000,2023.000000,10.000000,2.000000,4.000000,0.0,0.000000,0.0,190.000000,137.000000,1995.000000,1.000000,1.000000,2024-02-19 00:00:00,2024-05-06 00:00:00,2027-07-16 00:00:00,0.000000e+00,NaN,5.000000,2024-02-07 00:00:00,2024-02-28 00:00:00,2.163450e+04,0.0,1.230314e+06,127.000000,2.377125e+04,0.000000,6.292749e-02,0.000000,0.950043,0.0,6.094278e-03,2.583623e-02,2.188531e-02
max,72.000000,9.960005e+06,17140.000000,1.0,9.999999e+06,17140.000000,0.0,0.0,0.0,5.000000,23.000000,20075.000000,3500.000000,19855.000000,2033.000000,12.000000,2.000000,12.000000,

In [50]:
train_df_50 = pd.DataFrame()
for column, missing in train_df.isna().sum().items():
    #print(f"{column}: {(missing/len(train_df))*100}")
    if (missing/len(train_df))*100 > 50:
      print(f"{column}: {(missing/len(train_df))*100}")
      #train_df.drop(column, axis=1, inplace=True)

VARIANT: 97.67136699942466
RIM_KEY: 99.9818312119591
CERTIFICATE_TYPE: 99.98485934329925
FACTORY_NUMBER: 99.82335900515793
ENGINE_ID: 64.3760535373621
ENGINE_ID_ALT: 61.34590344298534
TRANSMISSION: 79.88109537604345
TRANSMISSION_ID: 83.56431246277923
RIMS: 99.93136235628994
FRONT_TIRES: 99.88089350062077
REAR_TIRES: 99.8849310090743
LEASING_CONTRACT_DATE: 88.9725550362871
LEASING_START: 82.73359509846475
LEASING_END: 77.93097879298685
PAINT_TYPE: 100.0
DEMONSTRATION_STATUS: 99.08853246661485
PRICE_LIST: 82.46510078630477
DAY_OF_REGISTRATION: 56.40803060431408


In [ ]:
threshold = 0.5 * len(train_df)  # 50% of the total rows
df_cleaned = train_df.dropna(thresh=threshold, axis=1)
df_cleaned.info()

In [58]:
std_dev = df_cleaned.std(numeric_only=True)  # Only numeric columns
print(std_dev)
columns_to_drop = std_dev[std_dev == 0].index
df_cleaned = df_cleaned.drop(columns=columns_to_drop)

COMPANY                               13.131775
MILEAGE                           187338.643199
OPERATING_HOURS                       56.329787
MILAGE_IN_FIELD                        0.000000
MILAGE_SALES                       96233.033557
OPERATING_HOURS_SALES                 56.329776
CERTIFICATE_TYPE_DATE                  0.000000
FRONT_TIRES_CONDITION                  0.000000
REAR_TIRES_CONDITION                   0.000000
NUMBER_DOORS                           2.161191
NUMBER_SEATS                           2.359111
PERMITTED_TOTAL_WEIGHT               662.282475
MAX_TRAILOR_LOAD                     364.058658
CURB_WEIGHT                          872.056447
YEAR_CONSTRUCTION                     26.109127
CONSTRUCTION_MONTH                     3.517440
NUMBER_AXLE                            0.907870
NUMBER_ENGINE_CYLINDER                 2.031409
REPAIR_RKZ                             0.000000
OPTICAL_CONDITION                      0.011095
TECHNICAL_CONDITION                    0

In [59]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99071 entries, 0 to 99070
Data columns (total 80 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   RPAKREP_VEHICLE_HKEY            99071 non-null  object        
 1   COMPANY                         99040 non-null  float64       
 2   OFFICE                          99006 non-null  object        
 3   OFFICE_MAIN_BRAND               96928 non-null  object        
 4   CHASSIS_NUMBER                  99040 non-null  object        
 5   MANUFACTURER_SHORT              99008 non-null  object        
 6   MANUFACTURER                    98987 non-null  object        
 7   VEHICLE_GROUP                   98931 non-null  object        
 8   VEHICLE_TYPE                    98963 non-null  object        
 9   MODEL_CODE                      76873 non-null  object        
 10  MILEAGE                         99040 non-null  float64       
 11  OP

In [60]:
df_cleaned.describe()

,COMPANY,MILEAGE,OPERATING_HOURS,MILAGE_SALES,OPERATING_HOURS_SALES,NUMBER_DOORS,NUMBER_SEATS,PERMITTED_TOTAL_WEIGHT,MAX_TRAILOR_LOAD,CURB_WEIGHT,YEAR_CONSTRUCTION,CONSTRUCTION_MONTH,NUMBER_AXLE,NUMBER_ENGINE_CYLINDER,OPTICAL_CONDITION,HORSEPOWER,KW,CCM,NUMBER_OWNERS,IS_USED_CAR,LEASING_MILAGE,VEHICLE_MODEL_ID,PURCHASE_DATE,PURCHASE_BOOKING_DATE,PURCHASE_MILAGE,AT_LOCATION_SINCE,LAID_UP_TIME,MILAGE_SALE,OPERATION_HOURS_SALE,SCALED_CURRENT_VALUE,SCALED_INVENTURAL_VALUE,SCALED_REPORT_VALUE,SCALED_GUIDE_PRICE,SCALED_TOTAL_SALES_PRICE_BASIS,SCALED_TOTAL_SALE_PRICE
count,99040.000000,9.904000e+04,99040.000000,9.904000e+04,99040.000000,99004.000000,99004.000000,99004.000000,99004.000000,99004.000000,74366.000000,70320.000000,99004.000000,99004.000000,73107.000000,99004.000000,99004.000000,99004.000000,99004.000000,99004.000000,9.900400e+04,99004.000000,97001,96173,9.900400e+04,9.900400e+04,99004.000000,9.900400e+04,99004.000000,9.900400e+04,99004.000000,99004.000000,9.900400e+04,9.900400e+04,9.900400e+04
mean,22.375808,3.071296e+04,0.275535,2.439775e+04,0.275374,3.432962,3.439932,243.442265,77.284766,1075.761202,2020.570059,6.360922,0.580775,2.544826,0.000041,149.995606,110.888772,1522.064715,0.569836,0.459699,7.046598e+03,3.010757,2023-09-04 06:56:46.752507648,2023-09-17 03:19:02.179197952,2.210805e+04,3.651454e+05,88.271939,2.422813e+04,0.275464,1.010119e+03,0.013350,0.944832,2.020125e+03,1.010080e+03,1.010078e+03
min,10.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-119.000000,0.000000,0.000000,0.000000e+00,0.000000,2013-07-24 00:00:00,2013-08-05 00:00:00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,2.980100e-04
25%,12.000000,1.000000e+01,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2020.000000,3.000000,0.000000,0.000000,0.000000,109.000000,81.000000,999.000000,0.000000,0.000000,0.000000e+00,0.000000,2023-04-19 00:00:00,2023-04-26 00:00:00,0.000000e+00,0.000000e+00,22.000000,0.000000e+00,0.000000,5.178134e-02,0.000000,0.950043,2.375720e-03,1.087726e-02,1.064459e-02
50%,18.000000,5.285000e+03,0.000000,3.003000e+03,0.000000,5.000000,5.000000,0.000000,0.000000,1345.000000,2022.000000,6.000000,0.000000,3.000000,0.000000,145.000000,110.000000,1498.000000,0.000000,0.000000,0.000000e+00,3.000000,2023-09-19 00:00:00,2023-10-05 00:00:00,3.000000e+00,0.000000e+00,59.000000,1.000000e+02,0.000000,5.704270e-02,0.000000,0.950043,4.373960e-03,1.805227e-02,1.578809e-02
75%,33.000000,2.694025e+04,0.000000,2.400000e+04,0.000000,5.000000,5.000000,0.000000,0.000000,1748.000000,2023.000000,10.000000,2.000000,4.000000,0.000000,190.000000,137.000000,1995.000000,1.000000,1.000000,0.000000e+00,5.000000,2024-02-07 00:00:00,2024-02-28 00:00:00,2.163450e+04,1.230314e+06,127.000000,2.377125e+04,0.000000,6.292749e-02,0.000000,0.950043,6.094278e-03,2.583623e-02,2.188531e-02
max,72.000000,9.960005e+06,17140.000000,9.999999e+06,17140.000000,5.000000,23.000000,20075.000000,3500.000000,19855.000000,2033.000000,12.000000,2.000000,12.000000,3.000000,13067.000000,9611.000000,98700.000000,9.000000,1.000000,2.000016e+06,9.000000,2024-08-13 00:00:00,2024-08-13 00:00:00,9.000009e+06,1.260104e+06,8782.000000,9.999999e+06,17140.000000,1.000000e+08,0.871569,0.963855,1.000000e+08,1.000000e+08,1.000000e+08
std,13.131775,1.873386e+05,56.329787,9.623303e+04,56.329776,2.161191,2.359111,662.282475,364.058658,872.056447,26.109127,3.517440,0.907870,2.031409,0.011095,107.488948,78.410067,2533.676627,0.721395,0.498376,2.634628e+04,2.617623,NaN,NaN,7.355878e+04,5.631298e+05,103.060790,1.008774e+05,56.340016,3.178144e+05,0.031157,0.030980,4.494552e+05,3.178144e+05,3.178144e+05
